# Data Loader notebook

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from wrds import Connection

db = Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


## Querying S&P500 options data

In [13]:
start_year = 2000
end_year = 2023

boundary_filters = {
    start_year: f"AND a.date BETWEEN '{start_year}-08-31' AND '{start_year}-12-31'",
    end_year: f"AND a.date BETWEEN '{end_year}-01-01' AND '{end_year}-08-31'"
}

dfs = []

for year in tqdm(range(start_year, end_year + 1), desc="Loading options"):
    date_filter = boundary_filters.get(year, "")
    query = f"""
        SELECT
            a.date,
            a.exdate,
            a.cp_flag,
            a.strike_price / 1000.0 AS strike_price,
            a.best_bid,
            a.best_offer,
            a.impl_volatility,
            a.delta,
            (a.exdate - a.date) AS time_to_maturity
        FROM optionm.opprcd{year} a
        WHERE
            a.secid = 108105
            {date_filter}
            AND a.impl_volatility IS NOT NULL
            AND a.impl_volatility > 0
            AND a.impl_volatility < 1
            AND a.best_bid > 0
            AND a.best_offer > a.best_bid
            AND (a.exdate - a.date) >= 7
            AND (a.exdate - a.date) <= 1825
        ORDER BY a.date, a.exdate
    """
    df_year = db.raw_sql(query, date_cols=["date", "exdate"])
    dfs.append(df_year)

options = pd.concat(dfs, ignore_index=True)

Loading options: 100%|██████████| 24/24 [05:57<00:00, 14.91s/it]


## Querying S&P500 price data

In [14]:
dfs_price = []

for year in tqdm(range(start_year, end_year + 1), desc="Loading index"):
    if year == start_year:
        date_filter = f"AND date BETWEEN '{year}-08-31' AND '{year}-12-31'"
    elif year == end_year:
        date_filter = f"AND date BETWEEN '{year}-01-01' AND '{year}-08-31'"
    else:
        date_filter = ""

    query = f"""
        SELECT date, close AS price
        FROM optionm.secprd{year}
        WHERE secid = 108105
        {date_filter}
        ORDER BY date
    """
    df_price_year = db.raw_sql(query, date_cols=["date"])
    dfs_price.append(df_price_year)

gspc = pd.concat(dfs_price, ignore_index=True)


Loading index: 100%|██████████| 24/24 [00:01<00:00, 19.33it/s]


## Querying risk-free rate

In [15]:
query = """
    SELECT date, rf
    FROM ff.factors_daily
    WHERE date BETWEEN '2000-08-31' AND '2023-08-31'
    ORDER BY date
"""

rf_daily = db.raw_sql(query, date_cols=["date"])
rf_daily["rf_decimal"] = rf_daily["rf"] / 100

## Merge datasets

In [16]:
df = (
    options
    .merge(gspc, on="date", how="left")
    .merge(rf_daily[["date", "rf_decimal"]], on="date", how="left")
)

df = df[[
    "date",
    "cp_flag",
    "strike_price",
    "price",
    "impl_volatility",
    "time_to_maturity",
    "rf_decimal"
]]

## Moneyness Calculation

$$
M=\left(\frac{1}{\sqrt{\tau}}\right)\log\left(\frac{F_{t, \tau}}{K}\right)
$$


where:
- $K$ is the strike price
- $\tau$ is the time to maturity in days
- $F_{t, \tau}$ is the forward price of SP500 with maturity $\tau$:
$$
F_{t, \tau} = S_te^{r\tau}
$$
with $r$ the risk-free rate

In [17]:
df["YTM"] = df["time_to_maturity"] / 365.0
df["rf"] = np.log(1 + df["rf_decimal"]) * df["YTM"]
df["F"] = df["price"] * np.exp(df["rf"])
df["M"] = (1 / np.sqrt(df["YTM"])) * np.log(df["F"] / df["strike_price"])

In [18]:
df.to_parquet("../data.nosync/data.parquet", index=False, engine="fastparquet")